# Cribbage Scoring Rules and Basic Objects

2021-07-03

This notebook will look at the basic scoring rules and the code solutions to those rules. This series of books is really about learning and automating the games scoring. The methods should allow you to input a hand and receive the proper score for the hand with a breakdown.

References:
- https://en.wikipedia.org/wiki/Cribbage
- https://www.pagat.com/adders/crib6.html
- https://boardgames.stackexchange.com/questions/24509/cribbage-scoring-rules-explanation

# Card Value

The value of each card is based on the rank of the card (there are some special scoring considerations). Ace is low, the other face cards are worth 10 points.

- A - The Ace is worth 1 point.
- J, Q, K - The Jack, Queen and King are equal and worth 10 points.
- All other cards are worth the face value of the rank.

# Scoring

The key elements that we score in a cribbage hand are:
- 15: Adding combination of cards that sum to 15 points based on the face values.
- pair: A pair of cards is worth 2 points. All combination of pairs are counted. For example if you have a triple (3 cards), that is worth 6 points as it yields 3 pairs from the combination of cars in the triple.
- run: Similar to a straight in poker. At least 3 cards in seqential order by rank. Receive 1 point per card in the sequence. Be mindful of combinations, for example 3,4,4,5 is two runs of 3,4,5 worth 6 points (as a run)
- flush: 4 or 5 cards (including the cut) with the same suit. When counting the crib, only 5 card flushes are valid.
- nobs: If the hand contains the Jack of that matches the suit of the cut card that is worth 1 point.
- nibs/heels: If the dealer cuts a Jack, the dealer receives 2 points.

# Conventions

For the code and the math we will use the following encoding conventions. The cards in a deck will be represented by two characters, the face value (A,2,3,4,5,6,7,8,9,10,J,Q,K) and the suit (S, H, D, C). So the King of Spades would be: `KS` and the Two of Diamonds would be: `2D`. Face cards have a point value of 10, the ace is 1 Point and all other cards represent their value.

The player has four cards in their hand and a cut card. 

The following hand would be 1 pair for 2 points:  

2D, 2S, KD, QD, 4C

# Code

In [1]:
%%javascript
//Disable autoscroll in the output cells
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [15]:
import random

from cribbage.cards import (
    Card,
    make_deck,
    display_hand,
    score_hand,
    score_hand_breakdown,
        
)

## Card

Originally, the Card object was defined using a standard Python Class. We have revisted this decision and use a [Dataclass][link1]. This simplifies things quite a bit and removes a lot of boiler plate. The class itself was reduced and distilled removing things that are not required. Moving methods out to separate methods.

Another big decision was removing the `Hand` class. This wasn't required and is replaced with a simple list of `Cards` and some supporting methods.

The following cells will demonstrate how to use the `Card` class.


[link1]:https://docs.python.org/3/library/dataclasses.html


In [7]:
# Define a card
c = Card(*'8S')

# NOTE: We can use the unpacking to properly unpack the string in the Card constructor. This makes creating a card
# trivial

# NOTE: The order of the string must be rank then suit. It will through an exception otherwise

print(f'Plain text:         {c}')
print(f'Fancy suit:         {c.cool_display()}')
print(f'Fancy unicode card: {c.cool_display(display_card=True)}')
print()
print(f'{c} is worth {c.value()}')

Plain text:         8S
Fancy suit:         8♠
Fancy unicode card: 🂨

8S is worth 8


The Card object also implements sorting. This means that you can sort a list of cards and it will sort the cards by suit, then by rank. This will group cards together by suit and then rank. Not quite in the order that most cribbage players would sort a hand.

In [14]:
cards = [Card(*c) for c in ('3S', '4C', '4D', '5H')]
print('Unsorted: ', display_hand(cards, cool=True))

print('Sorted:   ', display_hand(sorted(cards), cool=True))

Unsorted:  ['3♠', '4♣', '4♦', '5♥']
Sorted:    ['5♥', '4♦', '3♠', '4♣']


The unsorted hand would be how a cribbage player could organize the cards. The sorted hand sorts by suit then by rank.

# Scoring

There are quite a few methods that were defined to find all fifteens, runs, pairs, etc.. There isn't a point in going through the individual methods here. They will not be used individually. This section will outline how to score a hand properly using the correct methods.

The following code demonstrates creating a deck, sampling randomly from it and getting a scoring breakdown.

In [26]:
# Create a deck and take 5 cards at random. 

deck = make_deck()

hand = list(random.sample(deck, 5))

# extract the last card as the cut card
cut = hand[-1]

# exclude the cut card from the list
hand = hand[:-1]

results = score_hand_breakdown(
    hand,
    cut,
    include_nibs=False,
    five_card_flush=False,
    basic=False,
)

print("\n".join(results))

Hand = ['2♦', '7♦', '5♦', '2♣'] Cut = T♣
-----------------------
1 Fifteens for 2
1 Pairs for    2
0 Runs for     0
Flush for      0
Nobs for       0
Nibs for       0
-----------------------
Total:         4

Fifteens:

1 - ['5♦', 'T♣']

Pairs:

1, ['2♦', '2♣']



>NOTE: There is also the `score_hand` method which will return the score of the hand and not display the break down.